In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
import sys, os
import pickle
from matplotlib import pyplot as plt
from tenpy.tools.params import asConfig
import tenpy as tp
import tenpy.linalg.np_conserved as npc
from tenpy.networks import Site, MPS, MPO

/Users/hk/miniconda3/lib/python3.8/site-packages/tenpy/tools/optimization.py:317: UserWarning: Couldn't load compiled cython code. Code will run a bit slower.
  warnings.warn("Couldn't load compiled cython code. Code will run a bit slower.")


<font color='orange'>
In this note we show hot to construct a translation operator as MPO, only for the one-unit-cell system.
    
The tensor package we use here is TENPY.
</font>

<font color='orange'>
We use spin-1/2 for illustration. 

First, we define the "SITE" (local degrees of freedom) as SpinHalf.
</font>

In [5]:
"""
SO6 chain DMRG code compact version for HPC use. 

Puiyuen 240712-240726
"""
import numpy as np
import numpy.linalg as LA
from copy import deepcopy
from tenpy.models.model import CouplingModel, MPOModel
from tenpy.networks.site import SpinSite, FermionSite, Site
from tenpy.models.lattice import Chain
from tenpy.networks.mps import MPS
from tenpy.networks.mpo import MPO
import tenpy.linalg.np_conserved as npc
from tenpy.linalg.charges import LegCharge, ChargeInfo
from tenpy.algorithms import dmrg
from tenpy.tools.params import asConfig
import pickle

def oprs_on_ket(oprs_original, ket):
    """
    Inputs: 
        1. oprs_original, list of strings, the operators in the middle
        2. ket, list of strings, the ket

    Outputs:
        1. coe, the coefficient, 1 or -1 or 0
        2. ket, list of strings or zero(integer), the result of application of operators
    """

    oprs = deepcopy(oprs_original)
    assert len(oprs) != 0 #there must be at least one operator inside list oprs
    coe = 1

    while len(oprs) != 0:
        opr = oprs[-1]

        if opr.endswith('d'): #creation operator
            opr = opr[:-1] #cut the 'd' in the end
            if any(opr in _ for _ in ket): #if opr is already in ket
                ket = 0
                return 0, ket #return (0,0)
            else: #opr is not in ket
                ket.insert(0,opr)
        else: #annihilation operator
            if any(opr in _ for _ in ket): 
                index = ket.index(opr) #index is the number of particles 'before' opr
                coe *= (-1)**index
                ket.remove(opr)
            else:
                ket = 0
                return 0, ket #return (0,0)
        oprs = oprs[:-1] #cut the operator in oprs after application
    return coe, ket

def get_ket_from_6states(n):
    if n == 1:
        return ['1','2']
    elif n == 2:
        return ['1','3']
    elif n == 3:
        return ['1','4']
    elif n == 4:
        return ['2','3']
    elif n == 5:
        return ['2','4']
    elif n == 6:
        return ['3','4']
    else:
        raise("Out of 6 states. ")

def S_representation_matrix(alpha, beta):
    if type(alpha) != int or type(beta) != int or alpha>4 or alpha<1 or beta>4 or beta<1:
        raise("Check your alpha and beta. They must be 1,2,3,4. ")
    S_mat = np.zeros((6,6))
    for left in range(1,7):
        bra = get_ket_from_6states(left)
        oprs = [str(alpha)+'d', str(beta)]
        oprs.insert(0, bra[0])
        oprs.insert(0, bra[1]) #put the annihilation operators in the front of oprs
        for right in range(1,7):
            ket = get_ket_from_6states(right)
            coe, ket = oprs_on_ket(oprs, ket)
            if ket == []:
                S_mat[left-1, right-1] = coe
            elif ket == 0:
                S_mat[left-1, right-1] = 0
            else:
                raise('something wrong')
    if alpha==beta:
        S_mat -= (1/2)*np.diag([1,1,1,1,1,1])
    return S_mat

def gram_schmidt(A):
    Q, _ = LA.qr(A)
    return Q

def schmidt_to_cartan_subalgebra(S1,S2,S3):
    A = np.array([S1.flatten(), S2.flatten(), S3.flatten()])
    Q = gram_schmidt(A.T).T

    B1 = Q[0].reshape(6, 6)
    B2 = Q[1].reshape(6, 6)
    B3 = Q[2].reshape(6, 6)

    C1 = B1 * np.sqrt(2 / np.trace(B1 @ B1.T))
    C2 = B2 * np.sqrt(2 / np.trace(B2 @ B2.T))
    C3 = B3 * np.sqrt(2 / np.trace(B3 @ B3.T))

    return C1, C2, C3

def get_opr_list():
    su4g = dict()
    for alpha in range(1,5):
        for beta in range(1,5):
            su4g[(alpha,beta)] = S_representation_matrix(alpha,beta)


    so6g = [] #start with 15 generators
    coe_list = [] #the coefficient list, list of str
    for alpha in range(1,5):
        for beta in range(1,5):
            if not(alpha==4 and beta==4):
                so6g.append(S_representation_matrix(alpha,beta))
    C1, C2, C3 = schmidt_to_cartan_subalgebra(S_representation_matrix(1,1), S_representation_matrix(2,2), S_representation_matrix(3,3))
    so6g[0] = C1; so6g[5] = C2; so6g[10] = C3

    for a in range(1,5):
        for b in range(1,5):
            for c in range(1,5):
                for d in range(1,5):
                    SiSi = su4g[(a,b)] @ su4g[(c,d)]
                    Amat = np.zeros((36, len(so6g)))
                    B = SiSi.reshape(-1,1)
                    for l in range(len(so6g)):
                        Amat[:,l] = so6g[l].reshape(-1,1)[:,0]
                    #print("a,b,c,d",a,b,c,d,'shape of equation', Amat.shape, B.shape)
                    pcoe, resi, rank, sing = LA.lstsq(Amat, B, rcond=None)
                    #print("shape of coe", coe.shape)
                    if len(resi)!=0 and resi[0]>1e-10: #no solution
                        so6g.append(SiSi)
                        #print("a,b,c,d",a,b,c,d,"New added to so6g, now we have",len(so6g),'operators. ') #no more output here 240724
                        pcoe = np.append(np.zeros((len(so6g)-1, 1)),1).reshape(len(so6g),1)
                        coe_list.append(pcoe)
                    else:
                        coe_list.append(pcoe)
    
    so6g_new = deepcopy(so6g)

    for i in range(15,36):
        if i == 15:
            so6g_new[i] = np.diag([1,1,1,1,1,1])
        elif i == 16:
            so6g_new[i] = np.diag([2,-1,-1,-1,-1,2])/np.sqrt(6)
        elif i == 20:
            so6g_new[i] = np.diag([0,-1,1,1,-1,0])/np.sqrt(2)
        elif i in {17,18,19,21,22,23,26,31}:
            so6g_new[i] *= 2
        else:
            so6g_new[i] *= np.sqrt(2)

    coe_list_new = []

    for a in range(1,5):
        for b in range(1,5):
            for c in range(1,5):
                for d in range(1,5):
                    SiSi = su4g[(a,b)] @ su4g[(c,d)]
                    Amat = np.zeros((36, len(so6g_new)))
                    B = SiSi.reshape(-1,1)
                    for l in range(len(so6g_new)):
                        Amat[:,l] = so6g_new[l].reshape(-1,1)[:,0]
                    pcoe = LA.solve(Amat, B)
                    coe_list_new.append(pcoe)

    for i in range(len(coe_list_new)):
        coe_list_new[i] = coe_list_new[i].reshape(36)

    def pvec_new(a,b,c,d):
        return coe_list_new[64*(a-1)+16*(b-1)+4*(c-1)+d-1]

    cmn_new = np.zeros((36,36))

    P = dict()
    for a in range(1,5):
        for b in range(1,5):
            for c in range(1,5):
                for d in range(1,5):
                    P[(a,b,c,d)] = pvec_new(a,b,c,d)

    for m in range(36):
        for n in range(36):
            for a in range(1,5):
                for b in range(1,5):
                    for c in range(1,5):
                        for d in range(1,5):
                            cmn_new[m,n] += P[(a,b,c,d)][m] * P[(b,a,d,c)][n]

    return so6g_new, cmn_new

class SO6Site(Site):
    def __init__(self, so6g=None, cons_N=None, cons_S=None):
        if so6g is None:
            so6g, _ = get_opr_list()
        self.conserve = [cons_N, cons_S]
        self.cons_N = cons_N
        self.cons_S = cons_S
        self.so6g = so6g
        if cons_N == None and cons_S == 'U1':
            chinfo = npc.ChargeInfo([1, 1, 1], ['P', 'Q', 'R'])
            leg = npc.LegCharge.from_qflat(chinfo, [[-1, -2, 0], [-1, 1, 1], [-1, 1, -1], [1, -1, 1], [1, -1, -1], [1, 2, 0]])
        elif cons_N == 'N' and cons_S == 'U1':
            chinfo = npc.ChargeInfo([1, 1, 1, 1], ['FakeN', 'P', 'Q', 'R'])
            leg = npc.LegCharge.from_qflat(chinfo, [[1, -1, -2, 0], [1, -1, 1, 1], [1, -1, 1, -1], [1, 1, -1, 1], [1, 1, -1, -1], [1, 1, 2, 0]])
        elif cons_N == 'N' and cons_S == 'None':
            chinfo = npc.ChargeInfo([2], ['FakeN'])
            leg = npc.LegCharge.from_qflat(chinfo, [[1],[1],[1],[1],[1],[1]])
        else:
            print("No symmetry used in site SO6Site. ")
            leg = npc.LegCharge.from_trivial(6)
        
        ops = dict()
        for i in range(36):
            ops['lambda{}'.format(i)] = self.so6g[i]
        
        names = ['a', 'b', 'c', 'd', 'e', 'f'] #for the 6 double-occupied states we defined. 
        Site.__init__(self, leg, names, **ops)

    def __repr__(self):
        return "site for half-filled parton in 6 basis with conserve = {}".format([self.cons_N, self.cons_S])

<font color='orange'>
With SO6Site, the translation operators is obtained by trnslop_mpo
</font>

In [133]:
def trnslop_mpo(site, L=2, **kwargs):
    bc = kwargs.get('bc', 'pbc')    
    assert L>1
    leg = site.leg
    chinfo = leg.chinfo
    zero_div = [0]*chinfo.qnumber
    from tenpy.linalg.charges import LegPipe, LegCharge
    cleg = LegPipe([leg, leg.conj()], sort=False, bunch=False).to_LegCharge()
    nleg = npc.LegCharge.from_qflat(chinfo, [zero_div])
    
    swap = npc.zeros([leg, leg.conj(), leg, leg.conj()], qtotal=zero_div, labels=['p1', 'p1*', 'p2', 'p2*']) 
    for _i in range( site.dim ):
        for _j in range( site.dim ):
            swap[_j,_i,_i,_j] = 1
            
    reshaper = npc.zeros([leg, leg.conj(), cleg.conj()], qtotal=zero_div, labels=['p', 'p*', '(p*.p)'] )
    for _i in range( site.dim ):
        for _j in range( site.dim ):          
            idx = _i* site.dim + _j 
            reshaper[_i,_j,idx] = 1;
                
    swap = npc.tensordot(reshaper.conj(), swap, axes=((0,1),(0,1)))
    swap.ireplace_labels(['(p.p*)'], ['p1.p1*'])
    swap = npc.tensordot(swap, reshaper.conj(), axes=((1,2),(0,1)))
    swap.ireplace_labels(['(p.p*)'], ['p2.p2*'])
    
    left, right = npc.qr(swap)
        
    left  = npc.tensordot(reshaper, left, axes=((2), (0)))
    left.ireplace_labels([None], ['wR'])
    
    right = npc.tensordot(right, reshaper, axes=((1), (2)))
    right.ireplace_labels([None], ['wL'])
    
    bulk = npc.tensordot(right, left, axes=('p*','p'))
    if bc == 'pbc':
        bt = npc.zeros([nleg, leg, leg.conj()], qtotal=zero_div, labels=['wL', 'p', 'p*',])
        for _i in range( site.dim ):
            bt[0,_i,_i] = 1;
        left = npc.tensordot(bt, left, axes=(('p*', 'p')))
    
        bt = npc.zeros([leg, leg.conj(), nleg.conj()], qtotal=zero_div, labels=['p', 'p*', 'wR'])
        for _i in range( site.dim ):
            bt[_i,_i,0] = 1;
        right = npc.tensordot(right, bt, axes=(('p*', 'p')))    
    return [left] +  [bulk]*(L-2) +  [right] 

site = SO6Site(cons_S='U1')
trnslop_mpo(site, L=4)

[<npc.Array shape=(1, 6, 6, 36) labels=['wL', 'p', 'p*', 'wR']>,
 <npc.Array shape=(36, 6, 6, 36) labels=['wL', 'p', 'p*', 'wR']>,
 <npc.Array shape=(36, 6, 6, 36) labels=['wL', 'p', 'p*', 'wR']>,
 <npc.Array shape=(36, 6, 6, 1) labels=['wL', 'p', 'p*', 'wR']>]

In [134]:
def apply_mpo(mpo, mps, i0=0):
    L = len(mpo)
    for i in range( i0, i0+L ):
        B = npc.tensordot(mps.get_B(i, 'B'), mpo[i-i0], axes=('p', 'p*'))
        B = B.combine_legs([['wL', 'vL'], ['wR', 'vR']], qconj=[+1, -1])
        B.ireplace_labels(['(wL.vL)', '(wR.vR)'], ['vL', 'vR'])
        B.legs[B.get_leg_index('vL')] = B.get_leg('vL').to_LegCharge()
        B.legs[B.get_leg_index('vR')] = B.get_leg('vR').to_LegCharge()
        mps._B[i] = B#.itranspose(('vL', 'p', 'vR'))
    return mps

<font color='orange'>
We now check trnslop_mpo
</font>

In [137]:
"""
We shift S^z by one lattice site
"""
L = 4
site = SO6Site(cons_N='N', cons_S='U1')
swap = trnslop_mpo(site, L)
psi = MPS.from_product_state([site]*L, [0,3]*(L//2))
print( psi.expectation_value("lambda0") )
psis = psi.copy()
psis = apply_mpo(swap, psis)
psis.canonical_form()
print( psis.expectation_value("lambda0") )
print(psis.overlap(psi))

[-0.57735027  0.57735027 -0.57735027  0.57735027]
[ 0.57735027 -0.57735027  0.57735027 -0.57735027]
0.0


In [139]:
"""
k=pi
"""

psi = MPS.from_product_state([site]*L, [0,3]*(L//2))
psi = psi.add(MPS.from_product_state([site]*L, [3,0]*(L//2)), 1, -1)
psi.norm = 1
psi.canonical_form()
psis = psi.copy()
psis = apply_mpo(swap, psis)
psis.canonical_form()
print( (np.log(psis.overlap(psi)+0j)/np.pi/1j ).real, 'pi'  )

1.0 pi


In [140]:
"""
k=pi/2
"""

L = 4
site =  SO6Site(cons_N='N', cons_S='U1')
swap = trnslop_mpo(site, L)

psi = MPS.from_product_state([site]*L, [3,0,0,0])
psi = psi.add(MPS.from_product_state([site]*L, [0,3,0,0]), 1, 1j)
psi = psi.add(MPS.from_product_state([site]*L, [0,0,3,0]), 1, -1)
psi = psi.add(MPS.from_product_state([site]*L, [0,0,0,3]), 1, -1j)
psi.norm = 1
psi.canonical_form()
psis = psi.copy()
psis = apply_mpo(swap, psis)
psis.canonical_form()
print( (np.log(psis.overlap(psi)+0j)/np.pi/1j ).real, 'pi'  )

0.5 pi


<font color='orange'>
We consider a cylinder with length $l_x$ and circumference $l_y$. A pbc has been imposed along $l_y$. 

We will use translational operator to compute the momentum of schmidt vetors of reduced density matrix.
</font>

In [114]:
def calc_rdc_2mps(mps1, mps2, i=0):
    left = npc.ones( (mps1.get_B(0).legs[0], mps2.get_B(0).legs[0].conj()), labels=['vR', 'vR*'] )
    for idx in range(i):
        left = npc.tensordot(mps1.get_B(idx), left, axes=('vL', 'vR'))
        left = npc.tensordot(left, mps2.get_B(idx).conj(), axes=(('p', 'vR*'), ('p*', 'vL*')) )
    return left        

In [228]:
def calc_entanglement_spectrum_yshift(psi, lx, ly, i=None):
    if i is None:
        i = (lx//2)*ly
    site = psi.sites[0]
    swap = trnslop_mpo(site, ly)
    psis = psi.copy()
    for i0 in range(i//ly):
        psis = apply_mpo(swap, psis, i0=i0*ly)
    left = calc_rdc_2mps(psi, psis, i=i)
    l1, l2 = left.legs
    ees = []
    for _v1 in range(l1.block_number):
        cv1 = l1.charges[_v1]*l1.qconj
        qv1 = l1.get_qindex_of_charges(cv1)
        sv1 = l1.get_slice(qv1)
        cv2 = np.array(cv1)*-1
        qv2 = l2.get_qindex_of_charges(cv2)
        # print(cv1, cv2, qv1, qv2)
        block = left.get_block([qv1,qv2])
        eigval = np.linalg.eigvals(block)
        ees.append( (cv2, eigval ) )
    return ees

In [313]:
lx = 4
ly = 4
L = lx*ly
site = SpinHalf('N', '2*Sz')

psi = MPS.from_product_state([site]*L, [1,0,0,0]*lx)
psi = psi.add(MPS.from_product_state([site]*L, [0,1,0,0]*lx), 1, 1.j)
psi = psi.add(MPS.from_product_state([site]*L, [0,0,1,0]*lx), 1, -1)
psi = psi.add(MPS.from_product_state([site]*L, [0,0,0,1]*lx), 1, -1.j)

psi = psi.add(MPS.from_product_state([site]*L, [1,0,0,0,0,0,1,0]*(lx//2)), 1, 0.5)
psi = psi.add(MPS.from_product_state([site]*L, [0,1,0,0,0,0,0,1]*(lx//2)), 1, -0.5j)
psi = psi.add(MPS.from_product_state([site]*L, [0,0,1,0,1,0,0,0]*(lx//2)), 1, -0.5)
psi = psi.add(MPS.from_product_state([site]*L, [0,0,0,1,0,1,0,0]*(lx//2)), 1, 0.5j)

psi.norm = 1
psi.canonical_form()
ees = calc_entanglement_spectrum_yshift(psi, lx, ly,i=ly*2)
for _ in ees:
    print(_[0])
    eesC = -np.log(_[1])
    print(eesC.real)
    print(eesC.imag/np.pi)
print( psi.entanglement_spectrum(True)[3+ly] )

[8 4]
[1.60943791 1.60943791 1.60943791 1.60943791 2.99573227 2.99573227
 2.99573227 2.99573227]
[-5.00000000e-01  1.55300522e-16  1.00000000e+00  5.00000000e-01
 -0.00000000e+00 -5.00000000e-01  1.00000000e+00  5.00000000e-01]
[(array([8, 4]), array([1.60943791, 1.60943791, 1.60943791, 1.60943791, 2.99573227,
       2.99573227, 2.99573227, 2.99573227]))]
